In [ ]:
# Advanced Stock Price Prediction using Monte Carlo Simulation with Real Market Data
# This notebook implements a comprehensive stock price prediction model using historical data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set styling for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
# ==========================================
# 1. DATA ACQUISITION AND PREPROCESSING
# ==========================================

def fetch_stock_data(symbol, period="5y"):
    """
    Fetch historical stock data from Yahoo Finance
    
    Args:
        symbol (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
        period (str): Time period ('1y', '2y', '5y', 'max')
    
    Returns:
        pd.DataFrame: Historical stock data
    """
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period=period)
        data['Symbol'] = symbol
        print(f"Successfully fetched {len(data)} days of data for {symbol}")
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def calculate_returns(data):
    """
    Calculate various types of returns from price data
    
    Args:
        data (pd.DataFrame): Stock data with 'Close' column
    
    Returns:
        pd.DataFrame: Data with additional return columns
    """
    # Simple returns
    data['Daily_Return'] = data['Close'].pct_change()
    
    # Log returns (more suitable for Monte Carlo)
    data['Log_Return'] = np.log(data['Close'] / data['Close'].shift(1))
    
    # Rolling volatility (30-day window)
    data['Rolling_Volatility'] = data['Daily_Return'].rolling(window=30).std()
    
    # Remove NaN values
    data = data.dropna()
    
    return data



In [ ]:
# ==========================================
# 2. PARAMETER ESTIMATION
# ==========================================

def estimate_parameters(returns, method='historical'):
    """
    Estimate drift and volatility parameters from historical data
    
    Args:
        returns (pd.Series): Historical returns
        method (str): 'historical', 'ewma', or 'garch'
    
    Returns:
        tuple: (drift, volatility)
    """
    if method == 'historical':
        # Simple historical estimation
        drift = returns.mean()
        volatility = returns.std()
        
    elif method == 'ewma':
        # Exponentially Weighted Moving Average
        drift = returns.mean()
        # EWMA for volatility with decay factor
        weights = np.exp(np.linspace(-1, 0, len(returns)))
        weights /= weights.sum()
        volatility = np.sqrt(np.average((returns - drift)**2, weights=weights))
        
    else:
        # Default to historical
        drift = returns.mean()
        volatility = returns.std()
    
    return drift, volatility

def calculate_market_regime_parameters(data, regime_periods=[252, 126, 63]):
    """
    Calculate parameters for different market regimes
    
    Args:
        data (pd.DataFrame): Stock data with returns
        regime_periods (list): Different periods to analyze
    
    Returns:
        dict: Parameters for different regimes
    """
    regimes = {}
    
    for period in regime_periods:
        if len(data) >= period:
            recent_data = data.tail(period)
            drift, vol = estimate_parameters(recent_data['Log_Return'])
            regimes[f'{period}d'] = {'drift': drift, 'volatility': vol}
    
    return regimes


In [ ]:
# ==========================================
# 3. ADVANCED MONTE CARLO SIMULATION
# ==========================================

class AdvancedMonteCarlo:
    """
    Advanced Monte Carlo simulation for stock price prediction
    """
    
    def __init__(self, initial_price, drift, volatility):
        self.initial_price = initial_price
        self.drift = drift
        self.volatility = volatility
        self.paths = None
    
    def geometric_brownian_motion(self, days, num_simulations, dt=1/252):
        """
        Simulate stock prices using Geometric Brownian Motion
        
        Args:
            days (int): Number of days to simulate
            num_simulations (int): Number of simulation paths
            dt (float): Time step (default: 1/252 for daily steps)
        
        Returns:
            np.array: Simulated price paths
        """
        # Time grid
        time_steps = int(days / (dt * 252)) + 1
        
        # Initialize price paths
        paths = np.zeros((time_steps, num_simulations))
        paths[0] = self.initial_price
        
        # Generate random shocks
        dt_actual = dt
        drift_adj = (self.drift - 0.5 * self.volatility**2) * dt_actual
        vol_adj = self.volatility * np.sqrt(dt_actual)
        
        for t in range(1, time_steps):
            random_shocks = np.random.normal(0, 1, num_simulations)
            paths[t] = paths[t-1] * np.exp(drift_adj + vol_adj * random_shocks)
        
        self.paths = paths
        return paths
    
    def mean_reverting_simulation(self, days, num_simulations, mean_reversion_speed=0.1):
        """
        Simulate with mean reversion (Ornstein-Uhlenbeck process)
        
        Args:
            days (int): Number of days
            num_simulations (int): Number of paths
            mean_reversion_speed (float): Speed of mean reversion
        """
        dt = 1/252
        time_steps = days
        paths = np.zeros((time_steps, num_simulations))
        paths[0] = self.initial_price
        
        long_term_mean = self.initial_price * np.exp(self.drift * days/252)
        
        for t in range(1, time_steps):
            random_shocks = np.random.normal(0, 1, num_simulations)
            
            # Mean reversion component
            mean_revert = mean_reversion_speed * (long_term_mean - paths[t-1]) * dt
            
            # Volatility component
            vol_component = self.volatility * np.sqrt(dt) * random_shocks
            
            paths[t] = paths[t-1] + mean_revert + paths[t-1] * vol_component
        
        return paths
    
    def jump_diffusion_simulation(self, days, num_simulations, jump_intensity=0.1, jump_mean=-0.02, jump_std=0.05):
        """
        Simulate with jump diffusion (Merton model)
        
        Args:
            days (int): Number of days
            num_simulations (int): Number of paths
            jump_intensity (float): Average number of jumps per year
            jump_mean (float): Average jump size
            jump_std (float): Jump size standard deviation
        """
        dt = 1/252
        time_steps = days
        paths = np.zeros((time_steps, num_simulations))
        paths[0] = self.initial_price
        
        # Adjust drift for jumps
        adjusted_drift = self.drift - jump_intensity * (np.exp(jump_mean + 0.5 * jump_std**2) - 1)
        
        for t in range(1, time_steps):
            # Normal diffusion
            random_shocks = np.random.normal(0, 1, num_simulations)
            diffusion = (adjusted_drift - 0.5 * self.volatility**2) * dt + self.volatility * np.sqrt(dt) * random_shocks
            
            # Jump component
            jump_occurs = np.random.poisson(jump_intensity * dt, num_simulations)
            jump_sizes = np.random.normal(jump_mean, jump_std, num_simulations) * jump_occurs
            
            paths[t] = paths[t-1] * np.exp(diffusion + jump_sizes)
        
        return paths


In [ ]:
# ==========================================
# 4. ANALYSIS AND VISUALIZATION FUNCTIONS
# ==========================================

def plot_historical_analysis(data, symbol):
    """
    Create comprehensive historical analysis plots
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{symbol} - Historical Analysis', fontsize=16, fontweight='bold')
    
    # Price and volume
    ax1 = axes[0, 0]
    ax1.plot(data.index, data['Close'], color='blue', linewidth=2)
    ax1.set_title('Stock Price Over Time')
    ax1.set_ylabel('Price ($)')
    ax1.grid(True, alpha=0.3)
    
    # Daily returns
    ax2 = axes[0, 1]
    ax2.hist(data['Daily_Return'], bins=50, alpha=0.7, color='green', edgecolor='black')
    ax2.set_title('Distribution of Daily Returns')
    ax2.set_xlabel('Daily Return')
    ax2.set_ylabel('Frequency')
    ax2.axvline(data['Daily_Return'].mean(), color='red', linestyle='--', label='Mean')
    ax2.legend()
    
    # Rolling volatility
    ax3 = axes[1, 0]
    ax3.plot(data.index, data['Rolling_Volatility'], color='orange', linewidth=2)
    ax3.set_title('30-Day Rolling Volatility')
    ax3.set_ylabel('Volatility')
    ax3.grid(True, alpha=0.3)
    
    # QQ plot for normality
    ax4 = axes[1, 1]
    stats.probplot(data['Daily_Return'].dropna(), dist="norm", plot=ax4)
    ax4.set_title('Q-Q Plot (Normality Test)')
    
    plt.tight_layout()
    plt.show()

def plot_simulation_results(paths, actual_data, symbol, days_ahead):
    """
    Plot Monte Carlo simulation results with confidence intervals
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'{symbol} - Monte Carlo Simulation Results ({days_ahead} days ahead)', fontsize=16)
    
    # Main simulation plot
    ax1 = axes[0, 0]
    
    # Plot sample paths (subset for clarity)
    sample_paths = min(200, paths.shape[1])
    for i in range(0, sample_paths, 5):
        ax1.plot(paths[:, i], alpha=0.1, color='gray')
    
    # Calculate and plot percentiles
    percentiles = [5, 25, 50, 75, 95]
    colors = ['red', 'orange', 'blue', 'orange', 'red']
    labels = ['5th percentile', '25th percentile', 'Median', '75th percentile', '95th percentile']
    
    for i, (perc, color, label) in enumerate(zip(percentiles, colors, labels)):
        path_percentile = np.percentile(paths, perc, axis=1)
        ax1.plot(path_percentile, color=color, linewidth=2, label=label)
    
    ax1.set_title('Price Paths with Confidence Intervals')
    ax1.set_xlabel('Days')
    ax1.set_ylabel('Stock Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distribution of final prices
    ax2 = axes[0, 1]
    final_prices = paths[-1, :]
    ax2.hist(final_prices, bins=50, alpha=0.7, color='purple', edgecolor='black')
    ax2.axvline(np.mean(final_prices), color='red', linestyle='--', linewidth=2, label='Mean')
    ax2.axvline(np.median(final_prices), color='blue', linestyle='--', linewidth=2, label='Median')
    ax2.set_title('Distribution of Final Prices')
    ax2.set_xlabel('Final Price ($)')
    ax2.set_ylabel('Frequency')
    ax2.legend()
    
    # Probability distribution
    ax3 = axes[1, 0]
    current_price = actual_data['Close'].iloc[-1]
    returns = (final_prices / current_price - 1) * 100
    ax3.hist(returns, bins=50, alpha=0.7, color='green', edgecolor='black', density=True)
    ax3.set_title('Distribution of Returns (%)')
    ax3.set_xlabel('Return (%)')
    ax3.set_ylabel('Probability Density')
    ax3.axvline(0, color='red', linestyle='--', label='Break-even')
    ax3.legend()
    
    # Risk metrics
    ax4 = axes[1, 1]
    # Calculate VaR and CVaR
    var_95 = np.percentile(returns, 5)
    var_99 = np.percentile(returns, 1)
    cvar_95 = returns[returns <= var_95].mean()
    
    metrics = ['Mean Return', 'Std Dev', 'VaR (95%)', 'VaR (99%)', 'CVaR (95%)']
    values = [returns.mean(), returns.std(), var_95, var_99, cvar_95]
    
    bars = ax4.barh(metrics, values, color=['green' if v > 0 else 'red' for v in values])
    ax4.set_title('Risk Metrics (%)')
    ax4.set_xlabel('Value (%)')
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        ax4.text(value + (max(values) - min(values)) * 0.01, bar.get_y() + bar.get_height()/2, 
                f'{value:.2f}%', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def calculate_prediction_accuracy(historical_data, prediction_days=30):
    """
    Backtest the model by comparing predictions with actual outcomes
    """
    if len(historical_data) < prediction_days + 100:
        print("Insufficient data for backtesting")
        return None
    
    # Split data for backtesting
    train_data = historical_data.iloc[:-prediction_days]
    test_data = historical_data.iloc[-prediction_days:]
    
    # Estimate parameters from training data
    drift, volatility = estimate_parameters(train_data['Log_Return'])
    
    # Run simulation
    initial_price = train_data['Close'].iloc[-1]
    mc = AdvancedMonteCarlo(initial_price, drift, volatility)
    paths = mc.geometric_brownian_motion(prediction_days, 1000)
    
    # Calculate predicted prices
    predicted_mean = np.mean(paths, axis=1)
    predicted_std = np.std(paths, axis=1)
    
    # Actual prices
    actual_prices = test_data['Close'].values

    # Ensure both arrays have the same length (fix for the error)
    min_length = min(len(actual_prices), len(predicted_mean))
    actual_prices = actual_prices[:min_length]
    predicted_mean = predicted_mean[:min_length]
    predicted_std = predicted_std[:min_length]
    
    # Calculate metrics
    mae = mean_absolute_error(actual_prices, predicted_mean)
    rmse = np.sqrt(mean_squared_error(actual_prices, predicted_mean))
    mape = np.mean(np.abs((actual_prices - predicted_mean) / actual_prices)) * 100
    
    return {
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'actual': actual_prices,
        'predicted_mean': predicted_mean,
        'predicted_std': predicted_std
    }


In [ ]:
# ==========================================
# 5. MAIN EXECUTION
# ==========================================

def main_analysis(symbol='AAPL', prediction_days=60, num_simulations=5000):
    """
    Execute complete stock price prediction analysis
    """
    print(f"=== Stock Price Prediction Analysis for {symbol} ===")
    print(f"Prediction horizon: {prediction_days} days")
    print(f"Number of simulations: {num_simulations}")
    print("=" * 60)
    
    # 1. Fetch and preprocess data
    print("1. Fetching historical data...")
    data = fetch_stock_data(symbol, "5y")
    if data is None:
        return
    
    data = calculate_returns(data)
    print(f"   Data range: {data.index[0].date()} to {data.index[-1].date()}")
    
    # 2. Display basic statistics
    print("\n2. Basic Statistics:")
    current_price = data['Close'].iloc[-1]
    print(f"   Current Price: ${current_price:.2f}")
    print(f"   Annual Return (mean): {data['Daily_Return'].mean() * 252:.2%}")
    print(f"   Annual Volatility: {data['Daily_Return'].std() * np.sqrt(252):.2%}")
    
    # 3. Historical analysis visualization
    print("\n3. Generating historical analysis plots...")
    plot_historical_analysis(data, symbol)
    
    # 4. Parameter estimation for different regimes
    print("\n4. Parameter Estimation:")
    regimes = calculate_market_regime_parameters(data)
    for period, params in regimes.items():
        annual_return = params['drift'] * 252
        annual_vol = params['volatility'] * np.sqrt(252)
        print(f"   {period}: Return={annual_return:.2%}, Volatility={annual_vol:.2%}")
    
    # 5. Run Monte Carlo simulations
    print(f"\n5. Running Monte Carlo simulation ({num_simulations:,} paths)...")
    
    # Use the most recent regime (shortest period available)
    latest_params = regimes[list(regimes.keys())[0]]
    drift = latest_params['drift']
    volatility = latest_params['volatility']
    
    mc = AdvancedMonteCarlo(current_price, drift, volatility)
    
    # Standard Geometric Brownian Motion
    print("   - Geometric Brownian Motion simulation...")
    gbm_paths = mc.geometric_brownian_motion(prediction_days, num_simulations)
    
    # Mean reverting simulation
    print("   - Mean reverting simulation...")
    mr_paths = mc.mean_reverting_simulation(prediction_days, num_simulations)
    
    # Jump diffusion simulation
    print("   - Jump diffusion simulation...")
    jd_paths = mc.jump_diffusion_simulation(prediction_days, num_simulations)
    
    # 6. Results visualization
    print("\n6. Generating simulation results...")
    
    print("   - Standard Monte Carlo results:")
    plot_simulation_results(gbm_paths, data, f"{symbol} (Standard GBM)", prediction_days)
    
    # 7. Statistical summary
    print("\n7. Prediction Summary:")
    final_prices_gbm = gbm_paths[-1, :]
    
    print(f"   Current Price: ${current_price:.2f}")
    print(f"   Predicted Price Range:")
    print(f"     Mean: ${np.mean(final_prices_gbm):.2f}")
    print(f"     Median: ${np.median(final_prices_gbm):.2f}")
    print(f"     95% Confidence Interval: ${np.percentile(final_prices_gbm, 2.5):.2f} - ${np.percentile(final_prices_gbm, 97.5):.2f}")
    
    # Probability analysis
    prob_up = np.mean(final_prices_gbm > current_price) * 100
    prob_up_10 = np.mean(final_prices_gbm > current_price * 1.1) * 100
    prob_down_10 = np.mean(final_prices_gbm < current_price * 0.9) * 100
    
    print(f"\n   Probability Analysis:")
    print(f"     Price increases: {prob_up:.1f}%")
    print(f"     Price increases >10%: {prob_up_10:.1f}%")
    print(f"     Price decreases >10%: {prob_down_10:.1f}%")
    
    # 8. Backtesting (if enough data)
    print("\n8. Model Validation (Backtesting):")
    if len(data) > 100:
        backtest_results = calculate_prediction_accuracy(data, min(30, prediction_days))
        if backtest_results:
            print(f"   Mean Absolute Error: ${backtest_results['mae']:.2f}")
            print(f"   Root Mean Square Error: ${backtest_results['rmse']:.2f}")
            print(f"   Mean Absolute Percentage Error: {backtest_results['mape']:.2f}%")
    
    print("\n" + "=" * 60)
    print("Analysis Complete!")
    
    return {
        'data': data,
        'simulations': {
            'gbm': gbm_paths,
            'mean_reverting': mr_paths,
            'jump_diffusion': jd_paths
        },
        'parameters': {
            'drift': drift,
            'volatility': volatility,
            'current_price': current_price
        }
    }


In [ ]:
# ==========================================
# 6. EXECUTE ANALYSIS
# ==========================================

if __name__ == "__main__":
    # Run analysis for different stocks
    # You can change the symbol to any stock ticker (e.g., 'MSFT', 'GOOGL', 'TSLA', 'NVDA')
    
    results = main_analysis(
        symbol='AAPL',           # Stock symbol
        prediction_days=90,      # Prediction horizon
        num_simulations=10000    # Number of Monte Carlo paths
    )
    
    # Optional: Run analysis for multiple stocks
    symbols = ['AAPL', 'MSFT', 'GOOGL', 'TSLA']
    for symbol in symbols:
        print(f"\n{'='*80}")
        main_analysis(symbol, prediction_days=60, num_simulations=5000)


In [ ]:
# ==========================================
# 7. ADDITIONAL UTILITY FUNCTIONS
# ==========================================

def compare_models(symbol, prediction_days=30, num_simulations=1000):
    """
    Compare different Monte Carlo models side by side
    """
    print(f"Comparing Models for {symbol}")
    
    # Fetch data
    data = fetch_stock_data(symbol, "2y")
    data = calculate_returns(data)
    
    # Estimate parameters
    drift, volatility = estimate_parameters(data['Log_Return'])
    current_price = data['Close'].iloc[-1]
    
    # Initialize model
    mc = AdvancedMonteCarlo(current_price, drift, volatility)
    
    # Run different models
    gbm_paths = mc.geometric_brownian_motion(prediction_days, num_simulations)
    mr_paths = mc.mean_reverting_simulation(prediction_days, num_simulations)
    jd_paths = mc.jump_diffusion_simulation(prediction_days, num_simulations)
    
    # Plot comparison
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    models = [
        (gbm_paths, "Geometric Brownian Motion", axes[0]),
        (mr_paths, "Mean Reverting", axes[1]),
        (jd_paths, "Jump Diffusion", axes[2])
    ]
    
    for paths, title, ax in models:
        # Plot sample paths
        for i in range(0, min(100, num_simulations), 5):
            ax.plot(paths[:, i], alpha=0.1, color='gray')
        
        # Plot median path
        median_path = np.median(paths, axis=1)
        ax.plot(median_path, color='blue', linewidth=3, label='Median')
        
        # Plot confidence intervals
        upper_95 = np.percentile(paths, 95, axis=1)
        lower_5 = np.percentile(paths, 5, axis=1)
        ax.fill_between(range(len(upper_95)), lower_5, upper_95, alpha=0.3, color='lightblue')
        
        ax.set_title(title)
        ax.set_xlabel('Days')
        ax.set_ylabel('Price ($)')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle(f'{symbol} - Model Comparison', fontsize=16, y=1.02)
    plt.show()

    # First run the comparison to generate the paths
    result = compare_models('AAPL', prediction_days=60, num_simulations=2000)
    
    # Print final statistics
    models_data = [
        (gbm_paths, "GBM"),
        (mr_paths, "Mean Reverting"),
        (jd_paths, "Jump Diffusion")
    ]
    
    print("\nModel Comparison - Final Price Statistics:")
    print(f"{'Model':<15} {'Mean':<10} {'Median':<10} {'Std':<10} {'Min':<10} {'Max':<10}")
    print("-" * 70)
    
    for paths, name in models_data:
        final_prices = paths[-1, :]
        print(f"{name:<15} {np.mean(final_prices):<10.2f} {np.median(final_prices):<10.2f} "
              f"{np.std(final_prices):<10.2f} {np.min(final_prices):<10.2f} {np.max(final_prices):<10.2f}")


In [ ]:
# Example usage of model comparison
compare_models('AAPL', prediction_days=60, num_simulations=2000)